# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [7]:
import os
from io import BytesIO
import tempfile
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from pydub import AudioSegment
from pydub.playback import play

In [13]:
!pip install simpleaudio

In [14]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [15]:
system_message = "You are a helpful assistant for a programmer Airline called ProgrammerAI. "
system_message += "Give short, courteous answers, no more than 5 sentences. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [16]:
def tts(
    message: str,
    voice: str = "nova"
):
    """Send a TTS request to openai.

    :param message: The message.
    :param voice: Optional voice. See `https://www.openai.fm/` for voice options.
    """
    response = openai.audio.speech.create(
        model="tts-1",
        voice=voice,
        input=message
    )
    return response

In [18]:
def talker(message):
    # Set a custom directory for temporary files on Windows
    custom_temp_dir = os.path.expanduser("~/Documents/temp_audio")
    os.environ['TEMP'] = custom_temp_dir  # You can also use 'TMP' if necessary

    # Create the folder if it doesn't exist
    if not os.path.exists(custom_temp_dir):
        os.makedirs(custom_temp_dir)

    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )

    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    play(audio)

talker("Well hi there")

Input #0, wav, from '/var/folders/2l/c7fg4dt56j1gxp8ls1jx0lnr0000gn/T/tmpvhq2dul3.wav':
  Duration: 00:00:00.84, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s


In [33]:
from openai.types.chat import ChatCompletion


def chat(history):
    messages = [{"role": "system", "content": system_message}]
    for user, assistant in history:
        messages.append({"role": "user", "content": user})
        if assistant:
            messages.append({"role": "assistant", "content": assistant})

    response = openai.chat.completions.create(model=MODEL, messages=messages)
    reply = response.choices[0].message.content

    history.append((messages[-1]['content'], reply))
    # Uncomment if you want to use TTS or any other function
    talker(reply)
    return history

In [34]:
def do_entry(message, history):
    if history is None:
        history = []
    history.append((message, None))
    return "", history

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, label="AI Assistant")
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot])\
         .then(chat, inputs=chatbot, outputs=chatbot)

    clear.click(lambda: [], inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

/var/folders/2l/c7fg4dt56j1gxp8ls1jx0lnr0000gn/T/ipykernel_85378/1613301868.py:9: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500, label="AI Assistant")


* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.
